In [1]:
import IPython
import gym
from gym import envs
print envs.registry.all()[0]
import time
from SAM.SAM_Core import SAMCore
from SAM.SAM_Core import SAMDriver
import matplotlib.pyplot as plt
import pylab as pb
import sys
import operator
import pickle
import numpy as np
from scipy.spatial import distance
from ConfigParser import SafeConfigParser
from IPython.display import Audio
from IPython.display import HTML
from base64 import b64encode

path_to_audio = "/home/daniel/beep.wav"
audio_type = "wav"

sound = open(path_to_audio, "rb").read()
sound_encoded = b64encode(sound)
sound_tag = """
    <audio id="beep" controls src="data:audio/{1};base64,{0}">
    </audio>""".format(sound_encoded, audio_type)

play_beep = """
<script type="text/javascript">
    var audio = document.getElementById("beep");
    audio.play();
</script>
"""

HTML(sound_tag)

EnvSpec(Asteroids-ramDeterministic-v2)


IOError: [Errno 2] No such file or directory: '/home/daniel/beep.wav'

In [ ]:
pb.ion()
num_iters = 10
results = []

#PARAMETERS
enablePause = False
sleepTime = 0.1
#
samplingRate = 0.1
episodeLen = 1000

dataPointThresh = 1000
numberEpisodes = int(dataPointThresh*1.2)
tests = 50

model_type = 'mrd'
model_num_inducing = 40
model_num_iterations = 1000
model_init_iterations = 1200

recallMethod = 1
normalise = False

#prepare empty model
save_model=False
economy_save = True
visualise_output=True
modelAvailable = False
keepModel =False
verbose = False
exit = False

#prepare environment

classicControlList = ['MountainCar','CartPole','Acrobot','Pendulum']
AtariList = ['MsPacman','AirRaid','Alien','Amidar','Assault','Asterix','Asteroids','Atlantis','BankHeist','BattleZone','BeamRider']

env = gym.make(classicControlList[1]+'-v0')
env.tau = samplingRate
for i in range(num_iters):
    
    episodeCount = []
    mySAMpy = SAMDriver(False)
    mySAMpy.dataY = None
    mySAMpy.L = None
    cumulData = 0
    modelAvailable = False
    
    for i_episode in xrange(numberEpisodes):
        
        if(exit):
            exit = False
            break
            
        past_observation = env.reset()
        for t in xrange(episodeLen):
            env.render()

            if(modelAvailable):                          #recall from model
                if(recallMethod == 1):                   #Option 1: Find nearest observation
                    ret=mySAMpy.SAMObject.pattern_completion(past_observation[None,:])
                    mm = ret[0]
                    vv = ret[1]
                    post = ret[3] 

                    dists = np.zeros((mySAMpy.SAMObject.model.X.shape[0],1))

                    for j in range(dists.shape[0]):
                        dists[j,:] = distance.euclidean(mySAMpy.SAMObject.model.X.mean[j,:], mm[0].values)

                    nn, min_value = min(enumerate(dists), key=operator.itemgetter(1))

                    if mySAMpy.SAMObject.type == 'mrd':
                        textStringOut=mySAMpy.data_labels[int(mySAMpy.SAMObject.model.bgplvms[1].Y[nn,:])]

                    elif mySAMpy.SAMObject.type == 'bgplvm':
                        textStringOut=mySAMpy.data_labels[int(mySAMpy.L[nn,:])]

                    if(verbose):
                        print 'recall' , textStringOut

                    action = textStringOut[0]

                elif(recallMethod == 2):
                    action = env.action_space.sample()   #Option 2: check distance to nearest observation 
                                                         #and if greater than threshold sample new fantasy observation

            else:                                        #random action
                action = env.action_space.sample()
                if(verbose):
                    print action

            observation, reward, done, info = env.step(action)

            if done:
                #remove acion that led to this action
                np.delete(mySAMpy.dataY, len(mySAMpy.dataY)-1, 0)
                np.delete(mySAMpy.L, len(mySAMpy.L)-1, 0)

                episodeCount.append(t)
                if(verbose):
                    print "Episode finished after {} timesteps".format(t+1)
                cumulData += 1
                #check amount of data points collected if greater than threshold -> train
                if(cumulData > dataPointThresh):
                    if(modelAvailable):
                        exit = True
#                     ax =pb.plot(episodeCount)
#                     pb.draw()
#                     pb.pause(0.1)

                    #format data
                    if(normalise):
                        mySAMpy.Yn = mySAMpy.dataY
                    else:
                        mySAMpy.Yn = mySAMpy.dataY

                    if model_type == 'mrd':    
                        mySAMpy.X=None     
                        mySAMpy.Y = {'Y':mySAMpy.Yn,'L':mySAMpy.L}
                        mySAMpy.data_labels = mySAMpy.L.copy()
                    elif model_type == 'bgplvm':
                        mySAMpy.X=None     
                        mySAMpy.Y = {'Y':mySAMpy.Yn}
                        mySAMpy.data_labels = mySAMpy.L.copy()
                    #train model
                    if(not modelAvailable):
                        mySAMpy.training(model_num_inducing,model_num_iterations,model_init_iterations,'invertedPoleModel',save_model,economy_save,keepModel,None)
                        modelAvailable = True
                    cumulData -= tests
                    
                break
            else:
                if(mySAMpy.dataY == None):
                    mySAMpy.dataY = observation[None,:]
                else:
                    mySAMpy.dataY = np.vstack((mySAMpy.dataY,observation[None,:]))

                if(mySAMpy.L == None):
                    mySAMpy.L = action
                else:
                    mySAMpy.L = np.vstack((mySAMpy.L,action))

                past_observation = observation

                if(enablePause):
                    time.sleep(sleepTime)
    print len(episodeCount)
    
    results = results + episodeCount[-tests:]
    ax =pb.plot(results)
    pb.draw()
    pb.pause(0.1)
    
averageEpisodeLen = float(sum(results)) / float(len(results))
print averageEpisodeLen
env.render(close=True)
HTML(play_beep)

In [ ]:
averageEpisodeLen = float(sum(results)) / float(len(results))
print averageEpisodeLen